# Rule Splitting & ElasticSearch Uploads
## LMI Capstone Team
## Summer Chambers | Steve Morris | Kaleb Shikur

### Setup ES

In [ ]:
'''
Rule Splitting, Uploading to ES
'''
import pandas as pd
from elasticsearch import Elasticsearch, RequestsHttpConnection, ElasticsearchException
# from requests_aws4auth import AWS4Auth
import regex as re
from bs4 import BeautifulSoup
import numpy as np
import requests #gets urls
import time
# from aws_requests_auth.boto_utils import BotoAWSRequestsAuth

host = 'https://search-lmi-capstone-test-bbewbwsli4uchb7r24zujwrqne.us-east-1.es.amazonaws.com'
# auth = BotoAWSRequestsAuth(aws_host=host,
#                            aws_region='us-east-1',
#                            aws_service='es')
# awsauth = AWS4Auth(YOUR_ACCESS_KEY, YOUR_SECRET_KEY, REGION, 'es')

In [ ]:
es = Elasticsearch(host, timeout = 45)

# es = Elasticsearch(
#     hosts=[{'host': host, 'port': 443}],
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
print(es.info())

# es = Elasticsearch(
#     hosts=[{'host': host, 'port': 443}],
#     http_auth=auth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
# print(es.info())

### Define Rule Splits

In [ ]:
rule_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Rule/CMS-2018-0101-0001.txt"

In [ ]:
def splitRule_headers(rule_url):

    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    
    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    rulechunks = {}
    
    startdict = {'a2':['2. proposals for modified participation options under 5-year agreement periods', '3. creating a basic track with glide path to performance-based risk'], \
     'a3':['3. creating a basic track with glide path to performance-based risk', '4. permitting annual participation elections'], \
    'a4b':['b. proposals for permitting election of differing levels of risk within the basic track\'s glide path', 'c. proposals for permitting annual election of beneficiary assignment methodology'], \
    'a4c':['c. proposals for permitting annual election of beneficiary assignment methodology', '5. determining participation options based on medicare ffs revenue and prior participation '], \
    'a5b':['b. differentiating between low revenue acos and high revenue acos', 'c. determining participation options based on prior participation of aco legal entity and aco participants'], \
    'a5c':['c. determining participation options based on prior participation of aco legal entity and aco participants', 'd. monitoring for financial performance'], \
    'a5d': ['d. monitoring for financial performance', '6. requirements for aco participation in two-sided models'], \
    'a6b': ['b. election of msr/mlr by acos', 'c. aco repayment mechanisms'], \
    'a6c': ['c. aco repayment mechanisms', 'd. advance notice for and payment consequences of termination '], \
    'a6d2': ['(2) proposals for advance notice of voluntary termination', '(3) proposals for payment consequences of termination'], \
    'a6d3': ['(3) proposals for payment consequences of termination', '7. participation options for agreement periods beginning in 2019'], \
    'a7b': ['b. methodology for determining financial and quality performance for the 6-month performance years during 2019', 'c. applicability of program policies to acos participating in a 6-month performance year'], \
    'a7c': ['c. applicability of program policies to acos participating in a 6-month performance year', 'b. fee-for-service benefit enhancements'], \
    'b2a': ['a. shared savings program snf 3-day rule waiver', 'b. billing and payment for telehealth services'], \
    'b2b': ['b. billing and payment for telehealth services', 'c. providing tools to strengthen beneficiary engagement'], \
    'c2': ['2. beneficiary incentives', '3. empowering beneficiary choice'], \
    'c3a': ['3. empowering beneficiary choice', 'b. beneficiary opt-in based assignment methodology'], \
    'c3b': ['b. beneficiary opt-in based assignment methodology', 'd. benchmarking methodology refinements'],  \
    'd2': ['2. risk adjustment methodology for adjusting historical benchmark each performance year', '3. use of regional factors when establishing and resetting acos\' benchmarks'], \
    'd3b': ['b. proposals to apply regional expenditures in determining the benchmark for an aco\'s first agreement period', 'c. proposals for modifying the regional adjustment'], \
    'd3c': ['c. proposals for modifying the regional adjustment', 'd. proposals for modifying the methodology for calculating growth rates used in establishing, resetting, and updating the benchmark'], \
    'd3d': ['d. proposals for modifying the methodology for calculating growth rates used in establishing, resetting, and updating the benchmark', '4. technical changes to incorporate references to benchmark rebasing policies'], \
    'd4': ['4. technical changes to incorporate references to benchmark rebasing policies', 'e. updating program policies'], \
    'e2': ['2. revisions to policies on voluntary alignment', '3. revisions to the definition of primary care services used in beneficiary assignment'], \
    'e3': ['3. revisions to the definition of primary care services used in beneficiary assignment', '4. extreme and uncontrollable circumstances policies for the shared savings program'], \
    'e4': ['4. extreme and uncontrollable circumstances policies for the shared savings program', '5. program data and quality measures'], \
    'e5':['5. program data and quality measures', '6. promoting interoperability'], \
    'e6':['6. promoting interoperability', '7. coordination of pharmacy care for aco beneficiaries'], \
    'e7':['7. coordination of pharmacy care for aco beneficiaries', 'f. applicability of proposed policies to track 1+ model acos'], \
    'f2':['2. unavailability of application cycles for entry into the track 1+ model in 2019 and 2020', '3. applicability of proposed policies to track 1+ model acos through revised program regulations or revisions to track 1+ model participation agreements'], \
    'f3':['3. applicability of proposed policies to track 1+ model acos through revised program regulations or revisions to track 1+ model participation agreements', 'g. summary of proposed timing of applicability']}
    
    for key, value in startdict.items():    
       splitlist = splitlist.split(value[0]) #split on start of desired section
       split_further = splitlist[1].split(value[1]) #split again on start of undesired section
       rulechunks[key] = {"text": value[0]+split_further[0]} #choose only first half to upload to dict
       splitlist = splitlist[1] #choose second half to prepare for next split
    
    #print(rulechunks.keys())   
    #print(rulechunks["f3"])
    
    #lengths = [len(chunk) for chunk in rulechunks.values()]
    #print(lengths) #characters
    
    return rulechunks

In [31]:
#Updated Character-Count Split

def splitRule_line_chars(rule_url):
    #get rule
    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings

    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    #Split on new paragraphs
    paragraphs = splitlist.split('\r\n')
    
    #add new lines while under 2000 characters
    for i in range(len(paragraphs) - 1):
        while i < (len(paragraphs)-1) and len(paragraphs[i]) < 2000:
            paragraphs[i] += paragraphs[i+1]
            del(paragraphs[i+1])

    #add to dictionary
    rulechunks = {}
    keys = range(len(paragraphs))
    for i in keys:
        rulechunks[i] = {"text": paragraphs[i]}

    return rulechunks

In [36]:
splitchars = splitRule_line_chars(rule_url)
{key: len(value2) for key, value in splitchars.items() for key2, value2 in value.items()}

{0: 2091,
 1: 3044,
 2: 2163,
 3: 2416,
 4: 2980,
 5: 2273,
 6: 3614,
 7: 2980,
 8: 2509,
 9: 4233,
 10: 2080,
 11: 3151,
 12: 2861,
 13: 2215,
 14: 4077,
 15: 2288,
 16: 2522,
 17: 2135,
 18: 2047,
 19: 2558,
 20: 3697,
 21: 2178,
 22: 2305,
 23: 2416,
 24: 2346,
 25: 2673,
 26: 2146,
 27: 3033,
 28: 2975,
 29: 2506,
 30: 2007,
 31: 2561,
 32: 2146,
 33: 2642,
 34: 3371,
 35: 2082,
 36: 2015,
 37: 2263,
 38: 2460,
 39: 2339,
 40: 2241,
 41: 2957,
 42: 2177,
 43: 3418,
 44: 2397,
 45: 2587,
 46: 3469,
 47: 2117,
 48: 3567,
 49: 3916,
 50: 2464,
 51: 3333,
 52: 4572,
 53: 2043,
 54: 2627,
 55: 2966,
 56: 3302,
 57: 2574,
 58: 2355,
 59: 2690,
 60: 2199,
 61: 2686,
 62: 2331,
 63: 2453,
 64: 3027,
 65: 2086,
 66: 2953,
 67: 2649,
 68: 2213,
 69: 2270,
 70: 2924,
 71: 2498,
 72: 2171,
 73: 2841,
 74: 2464,
 75: 3105,
 76: 2194,
 77: 3386,
 78: 2355,
 79: 2074,
 80: 2448,
 81: 2214,
 82: 2285,
 83: 2400,
 84: 2203,
 85: 3050,
 86: 2542,
 87: 2818,
 88: 2310,
 89: 2740,
 90: 2133,
 91: 2143

In [ ]:
def splitRule_chars(rule_url):
    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    
    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    #chunk by num chars
    #chunklist = []
    
    chunklist = [(splitlist[i:i+8000]) for i in range(0, len(splitlist), 8000)]
    
    #make dict
    rulechunks = {}
    keys = range(len(chunklist))
    for i in keys:
        rulechunks[i] = {"text": chunklist[i]}

    return rulechunks

In [ ]:
def splitRule_headchars(rule_url):
    chunks = splitRule_headers(rule_url) #call first method
    for key, value in chunks.items():
        if len(value["text"]) > 50000: #check to see if chunk is really long
            smalldict = {}
            chunklist = [value["text"][i:i+25000] for i in range(0, len(value["text"]), 25000)] #split chunk
            keys = range(len(chunklist))
            for i in keys:
                smalldict[key+str(i)] = {"text": chunklist[i]} #add to sub-dictionary
            chunks[key] = smalldict #add to big dictionary
        else:
            chunks[key] = {key+str(0): value} #add to big dictionary
    return chunks

In [37]:
def splitRule_line_hybrid(rule_url):
    new_rule_chunks = {}
    chunks = splitRule_headers(rule_url)
    for key, value in chunks.items():
        for text, real_value in value.items():
            paragraphs = real_value.split('\r\n')
            #add new lines while under 2000 characters
            for i in range(len(paragraphs) - 1):
                while i < (len(paragraphs)-1) and len(paragraphs[i]) < 2000:
                    paragraphs[i] += paragraphs[i+1]
                    del(paragraphs[i+1])
            for i in range(len(paragraphs)):
                new_rule_chunks[key+str(i)] = {text: paragraphs[i]}
    return new_rule_chunks

In [47]:
result = splitRule_line_hybrid(rule_url)
lengths = {key: len(small_value) for key, value in result.items() for small_key, small_value in value.items()}

In [48]:
lengths

{'a20': 2370,
 'a21': 2861,
 'a22': 2215,
 'a23': 4077,
 'a24': 2288,
 'a25': 2522,
 'a26': 2135,
 'a27': 2047,
 'a28': 2558,
 'a29': 3697,
 'a210': 2178,
 'a211': 1810,
 'a30': 2014,
 'a31': 2148,
 'a32': 2907,
 'a33': 2352,
 'a34': 3688,
 'a35': 2975,
 'a36': 2506,
 'a37': 2007,
 'a38': 2561,
 'a39': 2146,
 'a310': 2642,
 'a311': 3371,
 'a312': 2082,
 'a313': 2015,
 'a314': 2263,
 'a315': 2460,
 'a316': 2339,
 'a317': 2241,
 'a318': 2957,
 'a319': 2177,
 'a320': 3418,
 'a321': 2397,
 'a322': 1031,
 'a4b0': 2144,
 'a4b1': 2230,
 'a4b2': 2117,
 'a4b3': 5,
 'a4c0': 3562,
 'a4c1': 3916,
 'a4c2': 2464,
 'a4c3': 3333,
 'a4c4': 4572,
 'a4c5': 2043,
 'a4c6': 2627,
 'a4c7': 2966,
 'a4c8': 3302,
 'a4c9': 2574,
 'a4c10': 2355,
 'a4c11': 2690,
 'a4c12': 2199,
 'a4c13': 2686,
 'a4c14': 2331,
 'a4c15': 2453,
 'a4c16': 3027,
 'a4c17': 2086,
 'a4c18': 2953,
 'a4c19': 2649,
 'a4c20': 2213,
 'a4c21': 2270,
 'a4c22': 2924,
 'a4c23': 2498,
 'a4c24': 2171,
 'a4c25': 2841,
 'a4c26': 2464,
 'a4c27': 3105,


### Uploading Rule Splits to ES

In [ ]:
def rulesplit_toES(rulechunks, es_index, idnum):
    for key, value in rulechunks.items():
            res = es.index(index=es_index, id=idnum, body={key:value}, doc_type='_doc')
            idnum += 1
            es.indices.refresh(index=es_index)
    print(f"Last id uploaded: {idnum-1}")

In [ ]:
def ruletoES_hybrid(rulechunks, es_index, idnum):
    for key, value in rulechunks.items():
        for small_key, small_value in value.items():
            res = es.index(index=es_index, id=idnum, body={small_key:small_value})
            idnum += 1
            es.indices.refresh(index=es_index)
    print(f"Last id uploaded: {idnum-1}")

In [ ]:
rulesplit_test = splitRule_headers(rule_url)
test_upload = rulesplit_toES(rulesplit_test, "headers_new", 1)

In [ ]:
rulesplit_test2 = splitRule_chars(rule_url)
test_upload2 = rulesplit_toES(rulesplit_test2, "char_8000", 1)

In [ ]:
rulesplit_test3 = splitRule_headchars(rule_url)
test_upload3 = ruletoES_hybrid(rulesplit_test3, "hybrid_new", 1)

### Querying ES

In [ ]:
#ES Search
def ES_search(index, querydict):
    search = es.search(index=index, doc_type="_doc", body={"query": querydict})
    test_list = {}
    if search['hits']['hits'] != []:
        for h in search['hits']['hits']:
            key = list((h['_source'].keys()))
            test_list[key[0]]=h['_score']
    return test_list

In [ ]:
test_query = {"simple_query_string": {"query": "urban"}}

In [ ]:
headers_test_search = ES_search("headers_new", test_query)
headers_test_search

In [ ]:
char_test_search = ES_search("char_8000", test_query)
char_test_search

In [ ]:
hybrid_test_search = ES_search("hybrid_new", test_query)
hybrid_test_search

### More Utilities:

In [ ]:
# Delete an index:
# es.indices.delete(index='index_name', ignore=[400, 404])

### Retrieve Comment Documents

In [ ]:
comment_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Comments/"

In [ ]:
def retrieve_comments(comment_url):
    comment_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Comments/"
    response = requests.get(comment_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    a_tags = soup.findAll("a")
    links = [tag["href"] for tag in a_tags]
    txt_links = [link for link in links if '.txt' in link]
    comments = {}
    for suffix in txt_links:
        comments[suffix] = requests.get(comment_url+suffix).text.lower()
        #print(f"scraping comment {suffix}")
    print(f"scraped {len(txt_links)} comments")
    return comments

In [ ]:
comments2018 = retrieve_comments(comment_url)

In [ ]:
comment_lengths = {key: len(value) for key, value in comments2018.items()}

In [ ]:
sorted_comment_lengths = dict(sorted(comment_lengths.items(), key=lambda x: x[1]))

In [ ]:
comments_short = {key: len(value) for key, value in comments2018.items() if "attach" in value and len(value) < 200}

In [ ]:
comments2018 = {(key[14:20]): value for key, value in comments2018.items()}
sorted_keys = sorted(list(comments2018.keys()))

# now for each key in the list
for i in range(len(comments2018)-1):
    # get key at index i and key at index i+1 and compare them
    if sorted_keys[i+1][0:4] == sorted_keys[i][0:4]:
        comments2018[sorted_keys[i+1]] = comments2018[sorted_keys[i]] + comments2018[sorted_keys[i+1]]
        del(comments2018[sorted_keys[i]])

#{key: len(value) for key, value in comments2018.items()}

In [ ]:
comments2018 = {key[0:4]:value for key, value in comments2018.items()}

In [ ]:
len(comments2018)

In [ ]:
import json
json = json.dumps(comments2018)
f = open('comments2018.json','w')
f.write(json)
f.close()

### Querying Comments

In [ ]:
es.indices.refresh(index='headers_new')

In [ ]:
es.indices.refresh(index='hybrid_new')

In [ ]:
test_comment001_query = {"simple_query_string": {"query": comments2018["0002"]}}

In [ ]:
ES_search("headers_new", test_comment001_query)

In [ ]:
ES_search("hybrid_new", test_comment001_query)

In [ ]:
# dict_first3 = {key:value for key,value in comments2018}
dict_first3 = {}
for key in list(comments2018.keys())[0:3]:
    dict_first3[key] = comments2018[key]

In [ ]:
# Test HYBRID on dictionary of 3 key/value

results = {}
for key, value in dict_first3.items():
    query =  {"simple_query_string": {"query": value}}
    try:
        search1 = ES_search("hybrid_new", query)
    except ElasticsearchException as es1:
            print(f'maximum clause error at index {key}')
    results[key] = list(search1.keys())
    time.sleep(1)
    es.indices.refresh(index='hybrid_new')
    print(i, end=' ')
    i+=1

In [ ]:
# Test HEADERS on dictionary of 3 key/value

results = {}
for key, value in dict_first3.items():
    query =  {"simple_query_string": {"query": value}}
    try:
        search1 = ES_search("headers_new", query)
    except ElasticsearchException as es1:
            print(f'maximum clause error at index {key}')
    results[key] = list(search1.keys())
    time.sleep(1)
    es.indices.refresh(index='headers_new')
    print(i, end=' ')
    i+=1

In [ ]:
# HYBRID Full dictionary Query

results = {}
i=0
for key, value in comments2018.items():
    query =  {"simple_query_string": {"query": value}}
    try:
        search1 = ES_search("hybrid_new", query)
    except ElasticsearchException as es1:
            print(f'maximum clause error at index {key}')
    results[key] = list(search1.keys())
    time.sleep(1)
    es.indices.refresh(index='hybrid_new')
    print(i, end=' ')
    i+=1

In [ ]:
# Capture HYBRID results in CSV
import csv
w = csv.writer(open('hybrid_results.csv', 'w'))
for key, val in results.items():
    w.writerow([key,val])

In [ ]:
# Capture HYBRID results in JSON
import json
json = json.dumps(results)
f = open('hybrid_results.json','w')
f.write(json)
f.close()

In [ ]:
# HEADERS Full dictionary Query

results = {}
i=0
for key, value in comments2018.items():
    query =  {"simple_query_string": {"query": value}}
    try:
        search1 = ES_search("headers_new", query)
    except ElasticsearchException as es1:
            print(f'maximum clause error at index {key}')
    results[key] = list(search1.keys())
    time.sleep(1)
    es.indices.refresh(index='headers_new')
    print(i, end=' ')
    i+=1

In [ ]:
import csv
w = csv.writer(open('header_results.csv', 'w'))
for key, val in results.items():
    w.writerow([key,val])

In [ ]:
import json
json = json.dumps(results)
f = open('header_results.json','w')
f.write(json)
f.close()

In [ ]:
results